This is a notebook for a simple traditional IDMRG algorithm with block growth method. 

When writing the julia code shown here, I refer to the python code written by Dr. Garrison and Dr. Mishmash from the website:
    
    https://simple-dmrg.readthedocs.io/en/latest/
    
In the following code, I DO NOT use good quantum numbers in the system.
    
For IDMRG calculation, we make use of the translation invariance of the system.
We gradually enlarge the system part and the environment part, until the physical quantity of every "cell" converge.

In [ ]:
using LinearAlgebra
using KrylovKit
using SparseArrays
Sz = sparse([0.5 0;0 -0.5])
Sp = sparse([0 1; 0 0])
Sm = sparse([0 0; 1 0])
phy_d = 2
struct block
    len ::Int
    basis_size ::Int
    ops ::Vector{Any}
end
function is_valid_block(Block)
    for op in Block.ops
        @assert size(op) == (Block.basis_size,Block.basis_size)
    end
    return true
end
function two_sites_enlarge(Sz1,Sp1,Sm1,Sz2,Sp2,Sm2)
    J = 1
    Jz = 1
    TwoSiteH = (J/2)*kron(Sp1,Sm2)+(J/2)*kron(Sm1,Sp2)+Jz*kron(Sz1,Sz2)
    return TwoSiteH
end
initial_block = block(1,phy_d,[spzeros(phy_d,phy_d),Sz,Sp,Sm])
function enlarge_block(b) #b is a block
    mblock = b.basis_size
    o = b.ops
    H = kron(o[1],diagm(ones(phy_d)))+kron(diagm(ones(mblock)),zeros(phy_d,phy_d))
    H = H + two_sites_enlarge(o[2],o[3],o[4],Sz,Sp,Sm)
    conn_Sz = kron(sparse(I,mblock,mblock),Sz)
    conn_Sp = kron(sparse(I,mblock,mblock),Sp)
    conn_Sm = kron(sparse(I,mblock,mblock),Sm)
    return block(b.len+1,b.basis_size*phy_d,[H,conn_Sz,conn_Sp,conn_Sm])
end
function ChangeBasis(operator,trans_mat)
    op_new = trans_mat'*operator*trans_mat
    return op_new
end

In [1]:
function single_dmrg_step(sys,env,m) #m is the bond dimension cutoff
    @assert is_valid_block(sys)
    @assert is_valid_block(env)
    sys_enl = enlarge_block(sys)
    if sys == env
        env_enl = sys_enl
    else
        env_enl = enlarge_block(env)
    end
    @assert is_valid_block(sys_enl)
    @assert is_valid_block(env_enl)
    
    m_sys_enl = sys_enl.basis_size
    m_env_enl = env_enl.basis_size
    sys_enl_op = sys_enl.ops
    env_enl_op = env_enl.ops
    superblock_H = kron(sys_enl_op[1],sparse(I,m_env_enl,m_env_enl))+kron(sparse(I,m_sys_enl,m_sys_enl),env_enl_op[1])+two_sites_enlarge(sys_enl_op[2],sys_enl_op[3],sys_enl_op[4],env_enl_op[2],env_enl_op[3],env_enl_op[4])
    vals,vecs,info = KrylovKit.eigsolve(superblock_H,1,:SR)
    energy = vals[1]
    psi0 = vecs[1]
    psi0 = reshape(psi0,(m_sys_enl,m_env_enl))
    psi0 = transpose(psi0)
    rho = Hermitian(psi0*psi0')
    sol = eigen(rho)
    evector = [sol.vectors[:,i] for i = 1:length(sol.values)]
    evalue = sol.values
    eigen_result = [i for i in zip(evalue,evector)]
    sort!(eigen_result,rev=true,by = x->x[1])
    #print(eigen_result)
    mm = min(m,length(eigen_result))
    trans_mat = zeros(ComplexF64,m_sys_enl,mm)
    for i = 1:mm
        trans_mat[:,i] = eigen_result[i][2]
    end
    truncation_error = 1-sum([x[1] for x in eigen_result[1:mm]])
    println("truncation error = $truncation_error")
    new_length = sys_enl.len
    new_size = mm
    new_H = ChangeBasis(sys_enl_op[1],trans_mat)
    new_sz = ChangeBasis(sys_enl_op[2],trans_mat)
    new_sp = ChangeBasis(sys_enl_op[3],trans_mat)
    new_sm = ChangeBasis(sys_enl_op[4],trans_mat)
    new_block = block(new_length,new_size,[new_H,new_sz,new_sp,new_sm])
    return new_block,energy
end
function infinite_system(L,m)
    b = initial_block
    while 2*(b.len) <L
        println("L = $(b.len*2+2)")
        b,energy = single_dmrg_step(b,b,m)
        println("E/L = $(energy/b.len/2)")
    end
end
L = 100
m = 20
infinite_system(L,m)

20